# 1. Limpieza de una noticia

Busca una noticia o crónica que tenga al menos 2000 palabras y realiza todos los pasos que hemos realizado hasta ahora.

https://ciencia.nasa.gov/universo/que-es-la-energia-oscura/

https://ciencia.nasa.gov/universo/historia-de-dos-telescopios-wfirst-y-hubble/

Guillermo Luigui Ubaldo Nieto Angarita

## 1.1 Cargar librerías

In [33]:
import re
import spacy
import unicodedata
import stanza
import nltk
import pdfminer
import pandas as pd
import os

from collections import Counter
from pdfminer.high_level import extract_text
from transformers import AutoTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from flair.data import Sentence
from flair.models import SequenceTagger
from huggingface_hub import login
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Introduce tu token aquí
token = os.getenv("FLAIR_TOKEN")

login(token=token)

# Ahora puedes cargar el modelo
# The model name has been changed from 'ner-spanish' to 'flair/ner-spanish-large'
tagger = SequenceTagger.load('flair/ner-spanish-large')

2025-08-25 12:43:14,722 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


## 1.2 Descargar los tokenizadores

In [3]:
nltk.download('punkt')
stanza.download('es')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2025-08-25 12:43:16 INFO: Downloaded file to C:\Users\guill\stanza_resources\resources.json
2025-08-25 12:43:16 INFO: Downloading default packages for language: es (Spanish) ...
2025-08-25 12:43:17 INFO: File exists: C:\Users\guill\stanza_resources\es\default.zip
2025-08-25 12:43:20 INFO: Finished downloading models and saved to C:\Users\guill\stanza_resources


In [4]:
nlp_stanza = stanza.Pipeline("es")
nlp_spacy = spacy.load("es_core_news_sm")  #cargar el modelo en español
tokenizer_bert = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

2025-08-25 12:43:20 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-08-25 12:43:21 INFO: Downloaded file to C:\Users\guill\stanza_resources\resources.json
2025-08-25 12:43:22 INFO: Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined          |
| mwt          | combined          |
| pos          | combined_charlm   |
| lemma        | combined_nocharlm |
| constituency | combined_charlm   |
| depparse     | combined_charlm   |
| sentiment    | tass2020_charlm   |
| ner          | conll02           |

2025-08-25 12:43:22 INFO: Using device: cpu
2025-08-25 12:43:22 INFO: Loading: tokenize
2025-08-25 12:43:22 INFO: Loading: mwt
2025-08-25 12:43:22 INFO: Loading: pos
2025-08-25 12:43:23 INFO: Loading: lemma
2025-08-25 12:43:24 INFO: Loading: constituency
2025-08-25 12:43:24 INFO: Loading: depparse
2025-08-25 12:43:25 INFO: Loading: sentiment
2025-08-25 12:43:25 INFO: Loading: ner
2025-08-25 12:43:26 INFO: Done loading processors!


## 1.3 Descargar Stopwords en Español

In [5]:
nltk.download('stopwords')
spanish_stopwords = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1.4 Cargar texto

In [6]:
file_path = "texts/que_es_la_energia_oscura_google_chrome_1.pdf"

## 1.5 Leer PDF

In [7]:
def read_pdf(file_path):
    return extract_text(file_path)

## 1.6 Eliminación de elementos no deseados

In [8]:
remove_texts = {
    "1 5   M I N   R E A D",
    "https://ciencia.nasa.gov/universo/que-es-la-energia-oscura/",
    "¿Qué es la energía oscura? - NASA Ciencia",
    """Equipo de redacción de Ciencia

JUN 10, 2024

ARTÍCULO

ÍNDICE DE CONTENIDOS

Una breve historia

Todo comenzó con las cefeidas

El descubrimiento de un universo en expansión

La expansión se está acelerando, según muestran las supernovas

¿Qué es exactamente la energía oscura?

El futuro
""",
"""
Compartir

Detalles

Ú L T I M A   A C T U A L I Z A C I Ó N Mar 27, 2025

Términos relacionados

Universo

Administración Nacional de
Aeronáutica y el Espacio

La NASA explora lo desconocido en el aire y el

espacio, hace innovaciones en beneficio de la

humanidad e inspira al mundo con sus

descubrimientos.

Acerca de la misión de la NASA

Join Us

Página de
inicio

Noticias y
eventos

Multimedia

NASA+ L I V E

Misiones

Seres
humanos en el
espacio

Aeronáutica

Tecnología

Tierra

El sistema
solar

El universo

Ciencia

Recursos de
aprendizaje

Acerca de la
NASA

NASA en
Español
""",
"""
Sigue a la NASA

Mapa del sitio   Para los medios de comunicación

Política de privacidad   FOIA   No FEAR Act

Otras cuentas sociales de
la NASA

Oficina del IG   Presupuesto e informes anuales

Boletín de la NASA

Informes financieros de la agencia   Comunícate con la NASA

Accesibilidad

Última actualización de la página: Mar 27, 2025

 Editor/a de esta página: Noelia González

Responsible NASA Official for Science: Diana Logreira
"""
}

def extract_text_without_unwanted_elements(text):
    for remove_text in remove_texts:
        text = re.sub(re.escape(remove_text), "", text, flags=re.DOTALL)
        text = text.lstrip("\n")

    text = re.sub(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', '', text)
    text = re.sub(r'\b\d{1,2}:\d{2}\s?(AM|PM)\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\d+/\d+\b', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

## 1.6 Conversión a minúsculas

In [9]:
def convertToLowerCase(text):
    return text.lower() 

## 1.7 Transformación UNICODE

In [10]:
def unicodeTransformation(text, method):
    return unicodedata.normalize(method, text) 

## 1.8 Tokenización

In [11]:
def spacy_tokenization(text):
    return [token.text for token in nlp_spacy(text)]

def nltk_tokenization(text):
    return nltk.word_tokenize(text, language='spanish')

def stanza_tokenization(text):
    doc = nlp_stanza(text)
    return [word.text for sent in doc.sentences for word in sent.words]

def bert_tokenization(text):
    max_length = 512
    tokens_bert = tokenizer_bert.tokenize(text)
    chunks = [tokens_bert[i:i+max_length] for i in range(0, len(tokens_bert), max_length)]
    tokens_flat = [tok for chunk in chunks for tok in chunk]
    return tokens_flat

## 1.9 Eliminación de stop words

In [12]:
# --- Function for NLTK ---
def remove_stopwords_nltk(text):
    """
    Return Spanish text without stop words using NLTK
    """
    spanish_stopwords = set(stopwords.words("spanish"))
    tokens = nltk.word_tokenize(text, language="spanish")
    filtered = [word for word in tokens if word.lower() not in spanish_stopwords]
    return " ".join(filtered)

In [13]:
# --- Function for spaCy ---
def remove_stopwords_spacy(text):
    """
    Return Spanish text without stop words using spaCy
    """
    nlp = spacy.load("es_core_news_sm")
    doc = nlp(text)
    filtered = [token.text for token in doc if not token.is_stop]
    return " ".join(filtered)

## 1.10 Lematización

In [14]:
# --- Function 1: Lemmatization with spaCy ---
def lemmatize_spanish(text):
    """
    Lemmatize Spanish text (no stopword removal)
    """
    doc = nlp_spacy(text)
    lemmas = [token.lemma_ for token in doc if token.is_alpha]
    return lemmas

In [15]:
def lemmatize_with_flair(text):
    """Lematizar texto usando Flair"""
    tagger = SequenceTagger.load('pos-fast')
    sentence = Sentence(text)
    tagger.predict(sentence)
    lemmas = [token.text for token in sentence.tokens]
    return lemmas

## 1.11 Stemming

In [16]:
# --- Function 2: Stemming with NLTK ---
def stem_spanish(text):
    """
    Stem Spanish text (no stopword removal)
    """
    stemmer = SnowballStemmer("spanish")
    tokens = nltk.word_tokenize(text, language="spanish")
    stems = [stemmer.stem(word) for word in tokens if word.isalpha()]
    return stems

## 1.12 Identificar NERs

In [17]:
def apply_ner(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
    entities = [(entity.text, entity.labels[0].value) for entity in sentence.get_spans('ner')]
    return entities

In [18]:
def count_entities(entities):
    counter = Counter(entities)
    data = [{'Entidad': ent, 'Tipo': typ, 'Frecuencia': freq} for (ent, typ), freq in counter.items()]
    df = pd.DataFrame(data)
    return df

## 1.13 Procedimiento de Textos

In [19]:
def text_processing_pipeline():
    text = read_pdf(file_path)

    text_unicode = unicodeTransformation(text, "NFKD")

    text_cleaned = extract_text_without_unwanted_elements(text_unicode)

    text_lower_case = convertToLowerCase(text_cleaned)

    text_without_stop_words = remove_stopwords_spacy(text_lower_case)

    tokens = stanza_tokenization(text_without_stop_words)
    
    return ' '.join(tokens)

## 1.14 Lematización y Stemming

In [20]:
clean_text = text_processing_pipeline()
lemmas = lemmatize_with_flair(clean_text)
stems = stem_spanish(clean_text)

Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBB

2025-08-25 12:43:42,483 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [31]:
clean_text_with_lemmas = " ".join(lemmas)
clean_text_with_stems = " ".join(stems)

In [21]:
lemma_counts = Counter(lemmas).most_common(100)
df_lemma = pd.DataFrame(lemma_counts, columns=['Lema', 'Frecuencia'])
print(df_lemma.head(100))

           Lema  Frecuencia
0       energia          43
1      universo          39
2        oscura          38
3   cientificos          23
4      galaxias          18
..          ...         ...
95    variables           2
96     estrella           2
97    brillante           2
98        tenue           2
99        midio           2

[100 rows x 2 columns]


In [22]:
stems_counts = Counter(stems).most_common(100)
df_stems = pd.DataFrame(stems_counts, columns=['Stems', 'Frecuencia'])
print(df_stems.head(100))

      Stems  Frecuencia
0    energi          43
1   univers          39
2     oscur          38
3   cientif          27
4    galaxi          18
..      ...         ...
95  humason           3
96  respald           3
97    fisic           3
98     llev           3
99    afuer           3

[100 rows x 2 columns]


In [23]:
entities = apply_ner(clean_text)
df_entities = count_entities(entities)
print(df_entities)

                   Entidad  Tipo  Frecuencia
0           cumulo pandora  MISC           1
1                  chandra  MISC           1
2    hubble telescopio vlt  MISC           1
3        telescopio subaru  MISC           1
4               hubble vlt  MISC           1
5                 big bang  MISC           1
6   henrietta swan leavitt   PER           1
7                  leavitt   PER           1
8                  slipher   PER           2
9                alexander   PER           1
10               friedmann   LOC           1
11         albert einstein   PER           1
12        georges lemaitre   PER           1
13               friedmann   PER           1
14                einstein   PER           8
15                lemaitre   PER           1
16            edwin hubble   PER           1
17          milton humason   PER           1
18                  hubble   PER           2
19                 humason   PER           1
20              ley hubble  MISC           1
21      le

## 1.15 Vectorización

### 1.15.1 Vectorización por TD-IDF

In [36]:
vec_tf_idf = TfidfVectorizer()

In [38]:
X = vec_tf_idf.fit_transform(lemmas)

In [39]:
vec_tf_idf.get_feature_names_out()[1:100]

array(['10000', '13800', '1912', '1917', '1919', '1922', '1927', '1929',
       '1990', '1998', '1a', '2011', '2021', '2023', '2025', '2027',
       '2744', '3d', '450', '683', '70', '85', 'abell', 'abril',
       'absoluta', 'absolutamente', 'absoluto', 'aceleracion',
       'acelerado', 'acelerando', 'acelerar', 'acerca', 'aclarar', 'acto',
       'actualidad', 'actualmente', 'adam', 'adicional', 'afirmaba',
       'afuera', 'agencia', 'albert', 'aleja', 'alejaban', 'alejado',
       'alejan', 'alejando', 'alexander', 'algun', 'ambas', 'anos',
       'anteriormente', 'antiguos', 'antiparticulas', 'aparecer',
       'apodada', 'apodo', 'apoyan', 'apoyando', 'apoyar', 'arcoiris',
       'areas', 'arrugas', 'articulo', 'astronoma', 'astronomicos',
       'astronomo', 'astronomos', 'atribuir', 'avances', 'ayudar', 'azul',
       'bang', 'basaba', 'belga', 'big', 'brian', 'brillante', 'brillo',
       'buscando', 'busqueda', 'cabo', 'calculado', 'cambiado', 'cambio',
       'campo', 'camp

In [40]:
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(1205, 631))

In [41]:
x_tf_idf_array = X.toarray()

## 1.16 Guardar Archivos

In [30]:
with open("texts/certificado_limpio.txt", "w", encoding="utf-8") as f:
    f.write(clean_text)

In [25]:
with open("texts/certificado_limpio_lammes.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(lemmas))

In [26]:
with open("texts/certificado_limpio_stems.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(stems))

In [27]:
df_lemma.to_excel('lemmas_top100.xlsx', index=False)

In [28]:
df_stems.to_excel('stems_top100.xlsx', index=False)

In [29]:
df_entities.to_excel('entities_top100.xlsx', index=False)